In [17]:
import itertools
import numpy
import urllib,json
import pickle
import sys,trace

RRR = RealField(100);
eps = var('eps');

#stolen from https://ask.sagemath.org/question/8390/creating-an-array-of-variables/
class VariableGenerator(object): 
     def __init__(self, prefix): 
         self.__prefix = prefix 
     @cached_method 
     def __getitem__(self, key): 
         return SR.var("%s%s"%(self.__prefix,key))

#object: what to dump; name: string without ".pickle"
def pickletofile(object,name):
    thefile = open(name+".pickle",'wb');
    pickle.dump(object,thefile);
    thefile.close();
    return

#filename: string without ".pickle"
def unpicklefile(filename):
    thefile = open(filename+".pickle",'rb');
    theobj = pickle.load(thefile);
    thefile.close();
    return theobj

def savewits():
    pickletofile(SC,"stoch_complexity");
    pickletofile(SW,"witnesses");
    return;

#NOTE: use dictionary NC for strings of length 2 through 10.
#expects sigma to be an array/list, like usual
def nfacomplexity(sigma):
    theurl = "http://math.hawaii.edu/~bjoern/complexity-api/?string="+''.join(str(ch) for ch in sigma);
    response = urllib.urlopen(theurl);
    data = json.loads(response.read());
    return int(data.get('complexity'))

#returns bit flip of a string (given as a tuple)
def flip(sigma):
    l = [];
    for ch in sigma:
        l.append(1-ch);
    return tuple(l)

#flips every string in an iterable, returns a Set
def flipset(strings):
    theset = [];
    for s in strings:
        theset.append(flip(s));
    return Set(theset)

#switches the order of a pair of matrices
def switchmats(matpair):
    return [matpair[1],matpair[0]]


#input: string sigma, given as an iterable; base ba
#output: decimal value (in QQ) of .sigma, where the digits of
#sigma are treated as being in base ba
def expbase(sigma,ba=2):
    sig = list(sigma);
    decstring = ".";
    for i in range(len(sig)):
        assert sig[i]<ba;
        decstring = decstring + str(sig[i]);
    return QQ(RR(decstring,base=ba));

#same as expbase but reverses sigma for you
def revexpbase(sigma,ba=2):
    sig = list(reversed(list(sigma)));
    decstring = ".";
    for i in range(len(sig)):
        assert sig[i]<ba;
        decstring = decstring + str(sig[i]);
    return QQ(RR(decstring,base=ba));

#expects start_states & accept_states to be lists
def accprob(transmatrices,sigma,start_states=None,accept_states=None):
    assert transmatrices[0].nrows() == transmatrices[0].ncols() == transmatrices[1].nrows() == transmatrices[1].ncols();
    n = transmatrices[0].nrows();
    Id = identity_matrix(n);
    if start_states == None:
        v = Matrix(Id[0]);
    else:
        v = Matrix(start_states);
    if accept_states == None:
        f = Matrix(Id[n-1]).transpose();
    else:
        f = Matrix(accept_states).transpose();
    A = v*Id;
    for ch in sigma:
        assert int(ch) == 0 or int(ch) == 1;
        A = A*transmatrices[int(ch)];
    A = A*f;
    return A[0][0];

#expects start_states & accept_states to be matrices already
def accprob_vector(transmatrices,sigma,start_states,accept_states):
    #assert transmatrices[0].nrows() == transmatrices[0].ncols() == transmatrices[1].nrows() == transmatrices[1].ncols() == len(accept_states);
    n = transmatrices[0].nrows();
    Id = identity_matrix(n);
    v = start_states;
    f = accept_states;
    A = v*Id;
    for ch in sigma:
        assert int(ch) == 0 or int(ch) == 1;
        A = A*transmatrices[int(ch)];
    A = A*f;
    return A[0][0];
 
def probmatrix(transmatrices,sigma):
    assert transmatrices[0].nrows() == transmatrices[0].ncols() == transmatrices[1].nrows() == transmatrices[1].ncols();
    A = identity_matrix(transmatrices[0].nrows());
    for ch in sigma:
        assert int(ch) == 0 or int(ch) == 1;
        A = A*transmatrices[int(ch)];
    return A;
    
#lengths can be either an integer or a list
def list_probs(transmatrices,start_states=None,accept_states=None,lengths=range(1,5)):
    assert transmatrices[0].nrows() == transmatrices[0].ncols() == transmatrices[1].nrows() == transmatrices[1].ncols();
    n = transmatrices[0].nrows();
    Id = identity_matrix(n);
    if start_states == None:
        v = Matrix(Id[0]);
    else:
        v = Matrix(start_states);
    if accept_states == None:
        f = Matrix(Id[n-1]).transpose();
    else:
        f = Matrix(accept_states).transpose();
    if isinstance(lengths,sage.rings.integer.Integer):
        lenlist = [lengths];
    else:
        lenlist = lengths;
    
    problist = []; #list of strings with their acceptance probabilities
    for i in lenlist:
        l = list(itertools.product([0,1],repeat=i));
        for st in l:
            A = v*Id;
            for ch in st:
                assert int(ch) == 0 or int(ch) == 1;
                A = A*transmatrices[int(ch)];
            A = A*f;
            problist.append((st,A[0][0]));
        
    return problist;

#pass a list of binary strings to test, instead of a range
def list_probs_fromwords(transmatrices,start_states,accept_states,words=None):
    assert transmatrices[0].nrows() == transmatrices[0].ncols() == transmatrices[1].nrows() == transmatrices[1].ncols() == len(accept_states);
    n = transmatrices[0].nrows();
    Id = identity_matrix(n);
    v = Matrix(start_states);
    f = Matrix(accept_states).transpose();
    if words == None:
        words = list(itertools.product([0,1],repeat=11));
    
    problist = []; #list of strings with their acceptance probabilities
    for st in words:
        A = v*Id;
        for ch in st:
            assert int(ch) == 0 or int(ch) == 1;
            A = A*transmatrices[int(ch)];
        A = A*f;
        problist.append((st,A[0][0]));
        
    return problist;
    
#expects a list in exactly the format returned by list_probs. if length>0, only looks at strings of exactly that length
def highest_prob(problist,length=0):
    assert length >= 0;
    if length == 0:
        sublist = problist;
    elif length > 0:
        sublist = [l for l in problist if len(l[0]) == length];
    highprob = max([l[1] for l in sublist]);
    highstr = [st for st in sublist if st[1] == highprob];
    return highstr;
    
def sortproblist(problist):
    return sorted(problist,key=lambda x:x[1],reverse=True);
    
    
#check transition matrices for rows summing to 1
def checkmatrices(P):
    assert P[0].nrows() == P[0].ncols() == P[1].nrows() == P[1].ncols();
    n = P[0].nrows();
    
    for i in range(2):
        for j in range(n):
            s = sum(P[i][j]);
            if s != 1:
                if s > 1: relto1 = " > 1";
                elif s < 1: relto1 = " < 1";
                print("P"+str(i)+" row "+str(j+1)+" sum is "+str(RRR(s))+relto1);
                
#return list of pairs of nstates x nstates matrices
#(1/step) will be the increment of transition probabilities
#if half=True, don't include (a) both ways to order a
#pair of matrices; (b) pairs where both mats are the same.
#if info=True, print sizes of stuff along the way
def bruteforce(nstates,step,half=False,info=False):
    linit = Compositions(nstates+step,length=nstates);
    if info: print(str(len(linit))+" partitions");
    #lrows = [ [0]*nstates ];
    lrows = []; #not including rows of all 0s anymore
    for t in linit:
        r = []; #do arithmetic on t by hand
        for i in t:
            r.append(QQ(i-1)/step);
        lrows.append(tuple(r));
    if info: print(str(len(lrows))+" rows");
    mats = Tuples(tuple(lrows),nstates);
    if info: print(str(len(mats.list()))+" matrices");
    if not half:
        mats2 = Tuples(tuple(mats),2);
    else:
        mats2 = list(itertools.combinations(mats,2));
    if info: print(str(len(mats2.list()))+" matrix pairs");
    Pnew = [];
    for P in mats2:
        Pnew.append([Matrix(QQ,P[0]),Matrix(QQ,P[1])]);
    return Pnew;

#do bruteforce() algo on only a given list of vectors (as lists)
#assumes all vectors are of the same length
#does NOT check vectors sum to 1
def bruteforcevecs(vecs,half=False,info=False):
    if info: print(str(len(vecs))+" rows");
    n = len(vecs[0]);
    mats = Tuples(vecs,n).list();
    if info: print(str(len(mats))+" matrices");
    if not half:
        mats2 = Tuples(mats,2).list();
    else:
        mats2 = list(itertools.combinations(mats,2));
    if info: print(str(len(mats2))+" matrix pairs");
    Pnew = [];
    for P in mats2:
        Pnew.append([Matrix(QQ,P[0]),Matrix(QQ,P[1])]);
    return Pnew;

#do bruteforce() algo using only the given list of transition
#probabilities as entries
#if useeps=True, the matrices will just be over SR
def bruteforceprobs(nstates,probs,half=False,info=False,useeps=False):
    for p in probs:
        if p<0 or p>1:
            probs.remove(p);
    pro = Set(probs);
    if info: print(str(pro.cardinality())+" possible entries");
    lrowsall = Tuples(pro.list(),nstates).list();
    lrows = [];
    for r in lrowsall:
        s = sum(r);
        if s == 1: lrows.append(r);
    if info: print(str(len(lrows))+" rows");
    mats = Tuples(lrows,nstates).list();
    if info: print(str(len(mats))+" matrices");
    if not half:
        mats2 = Tuples(mats,2).list();
    else:
        mats2 = list(itertools.combinations(mats,2));
    if info: print(str(len(mats2))+" matrix pairs");
    Pnew = [];
    if useeps:
        for P in mats2:
            Pnew.append([Matrix(P[0]),Matrix(P[1])]);
    else:
        for P in mats2:
            Pnew.append([Matrix(QQ,P[0]),Matrix(QQ,P[1])]);
    return Pnew;

#expects a list of pairs of matrices as returned by bruteforce()
#strlen = length of strings to try, accept_states = vector (list)
def finduniques(matpairs,strlen,start_states=None,accept_states=None):
    uniques = [];
    n = matpairs[0][0].nrows();
    if start_states == None:
        start_states = list(identity_matrix(n)[0]);
    if accept_states == None:
        accept_states = list(identity_matrix(n)[n-1]);
    words = list(itertools.product([0,1],repeat=strlen));
    for P in matpairs:
        probs = list_probs_fromwords(P,start_states,accept_states,words);
        high = highest_prob(probs);
        if len(high) == 1:
            uniques.append([P,high]);
    return uniques;

#updates PFA complexities and adds new witnesses to SW
#uniqlist should be in the format output by finduniques()
#ASSUMES ALL MATRICES IN uniqlist ARE OF THE SAME SIZE!!!!!
#specify the _same_ list of accept states for all of them.
#return number of new witnesses added
def addwits(uniqlist,start_states=None,accept_states=None):
    n = uniqlist[0][0][0].nrows();
    count = 0;
    if start_states == None:
        v = Matrix(identity_matrix(n)[0]);
    else:
        v = Matrix(start_states);
    if accept_states == None:
        f = Matrix(identity_matrix(n)[n-1]).transpose();
    else:
        f = Matrix(accept_states).transpose();
    for u in uniqlist:
        s = u[1][0][0];
        if not SC.has_key(s) or SC[s] > n:
            SC[s] = n;
        if SC[s] == n:
            try:
                ind = SW[s].index((u[0],v,f));
            except ValueError:
                SW[s].append((u[0],v,f));
                count = count + 1;
    return count;

#like addwits(), but only adds if there isn't a witness yet for
#each particular string
#uniqlist should be in the format output by finduniques()
#ASSUMES ALL MATRICES IN uniqlist ARE OF THE SAME SIZE!!!!!
#specify the _same_ list of accept states for all of them.
#return number of new witnesses added
def addwitsnew(uniqlist,start_states=None,accept_states=None):
    n = uniqlist[0][0][0].nrows();
    count = 0;
    for u in uniqlist:
        s = u[1][0][0];
        if len(SW[s]) > 0 and SC.has_key(s):
            continue;
        count = count + addonewit(u,s,start_states,accept_states);
    return count;

#adds thewit (a pair of matrices) as a witness for sigma to SW
#also updates SC if appropriate
#only does it if this witnesses minimal complexity
#return 0 if nothing was added or updated;
#       1 if only a new witness;
#       2 if new witness and complexity updated.
def addonewit(thewit,sigma,start_states=None,accept_states=None):
    assert len(thewit) == 2;
    n = thewit[0].nrows();
    assert n == thewit[1].nrows() == thewit[0].ncols() == thewit[1].ncols();
    ret = 0;
    if start_states == None:
        v = Matrix(identity_matrix(n)[0]);
    else:
        assert n == len(start_states);
        v = Matrix(start_states);
    if accept_states == None:
        f = Matrix(identity_matrix(n)[n-1]).transpose();
    else:
        assert n == len(accept_states);
        f = Matrix(accept_states).transpose();
    if not SC.has_key(sigma) or SC[sigma] > n:
        SC[sigma] = n;
        ret = ret + 1;
    if SC[sigma] == n:
        try:
            ind = SW[sigma].index((thewit,v,f));
        except ValueError:
            SW[sigma].append((thewit,v,f));
            ret = ret + 1;
    return ret;

#same as addonewit(), but start/accept_states should now already be a
#matrix in the correct form
def addonewit_vec(thewit,sigma,start_states,accept_states):
    n = thewit[0].nrows();
    ret = 0;
    if not SC.has_key(sigma) or SC[sigma] > n:
        SC[sigma] = n;
        ret = ret + 1;
    if SC[sigma] == n:
        try:
            ind = SW[sigma].index((thewit,start_states,accept_states));
        except ValueError:
            SW[sigma].append((thewit,start_states,accept_states));
            ret = ret + 1;
    return ret;
    

#among PFAs given by matpairs, returns the first found which
#has sigma as its highest-prob word among strings of len(sigma).
#assumes all matrices have the same size.
#if stop>0, find the first stop witnesses (if possible).
#if stop=0, find all witnesses, if possible.
def findwitness(matpairs,sigma,start_states=None,accept_states=None,stop=1):
    n = matpairs[0][0].nrows();
    if start_states == None:
        v = Matrix(identity_matrix(n)[0]);
    else:
        v = Matrix(start_states);
    if accept_states == None:
        f = matrix(identity_matrix(n)[n-1]).transpose();
    else:
        f = matrix(accept_states).transpose();
    words = list(itertools.product([0,1],repeat=len(sigma)));
    words.remove(sigma);
    
    found = 0;
    wits = [];
    for P in matpairs:
        sigprob = accprob_vector(P,sigma,v,f);
        if sigprob == 0:
            continue;
        iscandidate = True;
        for s in words:
            testprob = accprob_vector(P,s,v,f);
            if testprob >= sigprob:
                iscandidate = False;
                break;
        if iscandidate:
            if stop != 1:
                wits.append((P,v,f));
            elif stop == 1:
                return P; #the traditional behavior
            found = found + 1;
            if stop > 0 and found >= stop:
                return wits;
    if len(wits) == 0:
        return None;
    else:
        return wits;

#expects a list in the format given by finduniques
#sigma should be a TUPLE, not an array! (?)
#returns the first pair of matrices in uniqlist witnessing the
#PFA complexity of sigma
def firstwitness(uniqlist,sigma):
    return [u[0] for u in uniqlist if u[1][0][0] == sigma][0]

#expects a dictionary like SC or NC
#returns Set of keys in compdict with value=complexity
#restricts to length strlen if strlen>0; o/w no restr on length
#if less=True, returns keys with value<=complexity
def lookup(compdict,complexity,strlen=0,less=False):
    if strlen == 0 and not less:
        return Set([k for k in compdict.keys() if compdict[k] == complexity]);
    elif strlen > 0 and not less:
        return Set([k for k in compdict.keys() if compdict[k] == complexity and len(k) == strlen]);
    elif strlen == 0 and less:
        return Set([k for k in compdict.keys() if compdict[k] <= complexity]);
    elif strlen > 0 and less:
        return Set([k for k in compdict.keys() if compdict[k] <= complexity and len(k) == strlen]);


#return largest denominator in a pair of rational matrices
def matdenom(matpair):
    themax = 1;
    for i in range(2):
        for r in matpair[i]: #rows
            for e in r: #element
                d = QQ(e).denominator();
                themax = lcm(d,themax);
    return themax;

#return first witness for PFA complexity of sigma in SW with
#matdenom <= denom
def firstwitdenom(sigma,denom):
    if not SW.has_key(sigma) or len(SW[sigma]) == 0:
        return None;
    for mats in SW[sigma]:
        if matdenom(mats[0]) <= denom:
            return mats;
    return None;

#return list of witnesses for PFA complexity of sigma in SW
#with matdenom = denom
def witsdenom(sigma,denom):
    if not SW.has_key(sigma) or len(SW[sigma]) == 0:
        return None;
    thelist = [];
    for mats in SW[sigma]:
        if matdenom(mats[0]) == denom:
            thelist.append(mats);
    if thelist == []:
        return None;
    return thelist;

#return lowest denominator out of all witnesses for sigma in SW
def mindenom(sigma):
    if not SW.has_key(sigma) or len(SW[sigma]) == 0:
        return None;
    return min([matdenom(w[0]) for w in SW[sigma]]);

#return highest denominator out of all witnesses for sigma in SW
def maxdenom(sigma):
    if not SW.has_key(sigma) or len(SW[sigma]) == 0:
        return None;
    return max([matdenom(w[0]) for w in SW[sigma]]);

#determine if matpair represents an actual aut (as in Rabin)
def isactualaut(matpair):
    for i in range(2):
        for r in matpair[i]: #rows
            for e in r: #entries
                if e == 0:
                    return False;
    return True;

#substitute epses (tuple of epsilons, one tuple per matrix,
#one epsilon per row) into matpairs for the variable eps
def matsub(matpairs,epses):
    assert len(epses[0]) == len(epses[1]) == matpairs[0].nrows();
    newmat = [ [], [] ];
    for i in range(2):
        for e in range(len(epses[i])):
            newmat[i].append(matpairs[i][e].substitute(eps=epses[i][e]));
    return [Matrix(QQ,newmat[0]), Matrix(QQ,newmat[1])];

#just returns the digraph of the aut described by matpair
def autgraph(matpair):
    n = matpair[0].nrows();
    gr = DiGraph(n,loops=True,multiedges=True);
    for l in range(2): #letter seen
        for i in range(n): #transition from state i...
            for j in range(n): #to state j
                if matpair[l][i][j] > 0:
                    gr.add_edge(i,j,label=str(l)+" ("+str(matpair[l][i][j])+")");
    return gr;

#plots the specified aut from SW
#its=# of iterations for plotting algo
def autplotwit(sigma,ind,its=2):
    matpair = SW[sigma][ind][0];
    f = SW[sigma][ind][1];
    n = f.nrows();
    gr = autgraph(matpair);
    acc = [i for i in range(n) if f[i][0] == 1];
    notacc = Set(range(n)).difference(Set(acc)).list();
    return gr.plot(edge_labels=True,vertex_size=400,layout='spring',iterations=its,vertex_colors={'white': notacc,'yellow': acc});

#expects a matrix pair defining the automaton
def autplot(matpair,accept_states=None,its=2):
    n = matpair[0].nrows();
    if accept_states == None:
        f = Matrix(identity_matrix(n)[n-1]).transpose();
        acc = [n-1];
    else:
        f = Matrix(accept_states).transpose();
        acc = [i for i in range(n) if accept_states[i] == 1];
    gr = autgraph(matpair);
    notacc = Set(range(n)).difference(Set(acc)).list();
    return gr.plot(edge_labels=True,vertex_size=400,layout='spring',iterations=its,vertex_colors={'white': notacc,'yellow': acc});

#expects a digraph (probably the output of autgraph())
def autgraphplot(autg,accept_states=None,its=2):
    n = autg.order();
    if accept_states == None:
        f = Matrix(identity_matrix(n)[n-1]).transpose();
        acc = [n-1];
    else:
        f = Matrix(accept_states).transpose();
        acc = [i for i in range(n) if accept_states[i] == 1];
    notacc = Set(range(n)).difference(Set(acc)).list();
    return autg.plot(edge_labels=True,vertex_size=400,layout='spring',iterations=its,vertex_colors={'white': notacc,'yellow': acc});

#given PFA (as matrix pair and init/acc states) and string sigma, 
#print a list of all accepting paths for sigma (as a sequence of 
#states) and the probability accumulated along each path
#for simplicity, for now only works for a single start state
def pathlist(matpair,sigma,accept_states=None):
    n = matpair[0].nrows(); # #states
    l = len(sigma)+1; #length of path
    #if start_states == None:
    #    v = Matrix(identity_matrix(n)[0]);
    #else:
    #    v = Matrix(start_states);
    if accept_states == None:
        accept_states = matrix(identity_matrix(n)[n-1]).transpose().list();
    #list of all possible sequences of states of the correct len
    stateseqs = [];
    for i in range(n):
        if accept_states[i] == 1:
            for tup in Tuples(range(n),l-2):
                tupli = list(tup);
                tupli.insert(0,0);
                tupli.append(i);
                stateseqs.append(tupli);
    #list of all such paths with their probs
    seqsnprobs = [];
    for aseq in stateseqs:
        pathprob = 1;
        for k in range(l-1):
            pathprob = pathprob*matpair[sigma[k]][aseq[k]][aseq[k+1]];
        seqsnprobs.append((aseq,pathprob));
    for s in seqsnprobs:
        if s[1] > 0:
            print("path "+str(s[0])+" has prob "+str(s[1]));
            
#return true iff matpair has no rows of all zeros
def isvalidpfa(matpair):
    for mat in matpair:
        for r in mat:
            if sum(r) == 0:
                return False;
    return True;

In [2]:
#run immediately when restarting
SC = unpicklefile("stoch_complexity");
NC = unpicklefile("nfa_complexity");
SW = unpicklefile("witnesses");


S = {};
S[0] = Set([]);
for i in range(2,15):
    S[i] = Set(itertools.product([0,1],repeat=i));
    S[0] = S[0].union(S[i]);

In [ ]:
#STATS
print(len(SC))
for l in range(3,11):
    if l<9:
        top=4;
    else:
        top=5;
    for c in range(1,top):
        print("len "+str(l)+" complexity "+str(c)+": "+str(lookup(SC,c,l).cardinality())+" (NFA: "+str(lookup(NC,c,l).cardinality())+")")
    nu = lookup(SC,100,l,less=True).cardinality();
    if nu < 2^l:
        print("total classified "+str(nu)+"/"+str(2^l));
    witsum = sum([len(SW[k]) for k in SW.keys() if len(k)==l]);
    print("total witnesses "+str(witsum));
    print("");
print("# strings of each length with PFA=NFA complexity:");
for l in range(3,11):
    print(str(l)+": "+str(len([k for k in SC.keys() if len(k) == l and SC[k] == NC[k]])))
print("# strings of each length with PFA=NFA complexity + 1:")
for l in range(3,11):
    print(str(l)+": "+str(len([k for k in SC.keys() if len(k) == l and SC[k] == NC[k] + 1])))

In [18]:
tracer = trace.Trace(
    ignoredirs=[sys.prefix, sys.exec_prefix],
    trace=1,
    count=1);
tracer.run(bruteforce(2,2));
r = tracer.results()
r.write_results(show_missing=True, coverdir=".")

TypeError: unhashable type: 'list'

In [ ]:
test2x2 = bruteforce(2,2);
test2x3 = bruteforce(2,3);
test2x4 = bruteforce(2,4);
test2x5 = bruteforce(2,5);
test2x6 = bruteforce(2,6);
test2x7 = bruteforce(2,7);
test2x8 = bruteforce(2,8);
test3x2 = bruteforce(3,2);

somevecs = [ [1,0,0],[0,1,0],[0,0,1],[1/4,1/4,1/2],
           [1/4,1/2,1/4],[1/2,1/4,1/4]];
some3x4 = bruteforcevecs(somevecs,half=True);

somevecs2 = [ [1,0,0],[0,1,0],[0,0,1], [1/4,0,3/4],
             [1/4,3/4,0], [3/4,0,1/4], [3/4,1/4,0],
             [0,1/4,3/4], [0,3/4,1/4]];
some3x4_2 = bruteforcevecs(somevecs2,half=True);

In [29]:
half3x3 = bruteforce(3,3,half=True);
print len(half3x3);
#test2x10 = unpicklefile("test2x10");
test2x10 = bruteforce(2,10,half=True);
print len(test2x10);
#test2x15 = unpicklefile("test2x15");
#print len(test2x15);
#test2x20 = unpicklefile("test2x20");
test2x20 = bruteforce(2,20,half=True);
print len(test2x20);
#test4x1 = unpicklefile("test4x1");
#test3x4 = unpicklefile("test3x4");

499500
7260
83521
97020


In [20]:
moore = [Matrix([[1,0],[1/2,1/2]]), Matrix([[1/2,1/2],[0,1]])];
sortm = sortproblist(list_probs(moore,lengths=5,accept_states=(0,1)));
for w in sortm:
    if w[1] > 0:
        print("string "+str(w[0])+": "+str(w[1]))
        pathlist(moore,w[0]);
        print("")

string (1, 1, 1, 1, 1): 31/32
path [0, 0, 0, 0, 0, 1] has prob 1/32
path [0, 0, 0, 0, 1, 1] has prob 1/16
path [0, 0, 0, 1, 1, 1] has prob 1/8
path [0, 0, 1, 1, 1, 1] has prob 1/4
path [0, 1, 1, 1, 1, 1] has prob 1/2

string (0, 1, 1, 1, 1): 15/16
path [0, 0, 0, 0, 0, 1] has prob 1/16
path [0, 0, 0, 0, 1, 1] has prob 1/8
path [0, 0, 0, 1, 1, 1] has prob 1/4
path [0, 0, 1, 1, 1, 1] has prob 1/2

string (1, 0, 1, 1, 1): 29/32
path [0, 0, 0, 0, 0, 1] has prob 1/16
path [0, 1, 0, 0, 0, 1] has prob 1/32
path [0, 0, 0, 0, 1, 1] has prob 1/8
path [0, 1, 0, 0, 1, 1] has prob 1/16
path [0, 0, 0, 1, 1, 1] has prob 1/4
path [0, 1, 0, 1, 1, 1] has prob 1/8
path [0, 1, 1, 1, 1, 1] has prob 1/4

string (0, 0, 1, 1, 1): 7/8
path [0, 0, 0, 0, 0, 1] has prob 1/8
path [0, 0, 0, 0, 1, 1] has prob 1/4
path [0, 0, 0, 1, 1, 1] has prob 1/2

string (1, 1, 0, 1, 1): 27/32
path [0, 0, 0, 0, 0, 1] has prob 1/16
path [0, 0, 1, 0, 0, 1] has prob 1/32
path [0, 1, 1, 0, 0, 1] has prob 1/16
path [0, 0, 0, 0, 1, 1] h

In [23]:
moore3 = [Matrix(QQ,[[1,0],[2/3,1/3]]), Matrix(QQ,[[1/3,2/3],[0,1]])];
sort3 = sortproblist(list_probs(moore3,lengths=5,accept_states=(0,1)));
for w in sort3:
    if w[1] > 0:
        print "string "+str(w[0])+": "+str(w[1])+" "+str(revexpbase(w[0],3))
        pathlist(moore3,w[0]);
        print ""

string (1, 1, 1, 1, 1): 242/243 121/243
path [0, 0, 0, 0, 0, 1] has prob 2/243
path [0, 0, 0, 0, 1, 1] has prob 2/81
path [0, 0, 0, 1, 1, 1] has prob 2/27
path [0, 0, 1, 1, 1, 1] has prob 2/9
path [0, 1, 1, 1, 1, 1] has prob 2/3

string (0, 1, 1, 1, 1): 80/81 40/81
path [0, 0, 0, 0, 0, 1] has prob 2/81
path [0, 0, 0, 0, 1, 1] has prob 2/27
path [0, 0, 0, 1, 1, 1] has prob 2/9
path [0, 0, 1, 1, 1, 1] has prob 2/3

string (1, 0, 1, 1, 1): 236/243 118/243
path [0, 0, 0, 0, 0, 1] has prob 2/81
path [0, 1, 0, 0, 0, 1] has prob 8/243
path [0, 0, 0, 0, 1, 1] has prob 2/27
path [0, 1, 0, 0, 1, 1] has prob 8/81
path [0, 0, 0, 1, 1, 1] has prob 2/9
path [0, 1, 0, 1, 1, 1] has prob 8/27
path [0, 1, 1, 1, 1, 1] has prob 2/9

string (0, 0, 1, 1, 1): 26/27 13/27
path [0, 0, 0, 0, 0, 1] has prob 2/27
path [0, 0, 0, 0, 1, 1] has prob 2/9
path [0, 0, 0, 1, 1, 1] has prob 2/3

string (1, 1, 0, 1, 1): 224/243 112/243
path [0, 0, 0, 0, 0, 1] has prob 2/81
path [0, 0, 1, 0, 0, 1] has prob 8/243
path [0, 1,

In [26]:
for w in sort3:
    if w[1] != 2*revexpbase(w[0],3):
        print w,2*revexpbase(w[0],3)

In [22]:
moore3test = [Matrix(QQ,[[1,0],[1/3,2/3]]), Matrix(QQ,[[1/3,2/3],[0,1]])];
sort3t = sortproblist(list_probs(moore3test,lengths=5,accept_states=(0,1)));
for w in sort3t:
    if w[1] > 0:
        print "string "+str(w[0])+": "+str(w[1]*243)+"/243 "+str(2*revexpbase(w[0],3)*243)+"/243"
        pathlist(moore3test,w[0]);
        print ""

string (1, 1, 1, 1, 1): 242/243 242/243
path [0, 0, 0, 0, 0, 1] has prob 2/243
path [0, 0, 0, 0, 1, 1] has prob 2/81
path [0, 0, 0, 1, 1, 1] has prob 2/27
path [0, 0, 1, 1, 1, 1] has prob 2/9
path [0, 1, 1, 1, 1, 1] has prob 2/3

string (0, 1, 1, 1, 1): 240/243 240/243
path [0, 0, 0, 0, 0, 1] has prob 2/81
path [0, 0, 0, 0, 1, 1] has prob 2/27
path [0, 0, 0, 1, 1, 1] has prob 2/9
path [0, 0, 1, 1, 1, 1] has prob 2/3

string (1, 0, 1, 1, 1): 238/243 236/243
path [0, 0, 0, 0, 0, 1] has prob 2/81
path [0, 1, 0, 0, 0, 1] has prob 4/243
path [0, 0, 0, 0, 1, 1] has prob 2/27
path [0, 1, 0, 0, 1, 1] has prob 4/81
path [0, 0, 0, 1, 1, 1] has prob 2/9
path [0, 1, 0, 1, 1, 1] has prob 4/27
path [0, 1, 1, 1, 1, 1] has prob 4/9

string (0, 0, 1, 1, 1): 234/243 234/243
path [0, 0, 0, 0, 0, 1] has prob 2/27
path [0, 0, 0, 0, 1, 1] has prob 2/9
path [0, 0, 0, 1, 1, 1] has prob 2/3

string (1, 1, 0, 1, 1): 232/243 224/243
path [0, 0, 0, 0, 0, 1] has prob 2/81
path [0, 0, 1, 0, 0, 1] has prob 4/243
pat

In [5]:
s = list(reversed((1,1,1,1,0)));
thesum = QQ(0);
for i in range(len(s)):
    addnum = 2*s[i]*(2^i)/(3^(i+1));
    print "str "+str(s[i])+" add "+str(addnum)
    thesum = thesum + addnum
print thesum

str 0 add 0
str 1 add 4/9
str 1 add 8/27
str 1 add 16/81
str 1 add 32/243
260/243


In [ ]:
p,q,r,s=var('p,q,r,s');
mooresymb = [Matrix([[1,0],[p,q]]),Matrix([[r,s],[0,1]])];
#for astr in S[2].union(S[3]).union(S[4]):
for astr in [(0,1),(0,1,0),(0,1,1),(0,1,0,0),(0,1,0,1),
            (0,1,1,0),(0,1,1,1),
             (0,1,1,1,0),(0,1,1,1,1),(0,1,1,1,0,1)]:
            #(0,1,1,1,1),(0,1,1,0,1),(0,1,1,1)]:
    pretty_print(astr,probmatrix(mooresymb,astr))
    #print ""

In [39]:
#testing Bertoni's PFA
bert1 = [Matrix(QQ,[[1/4,3/4],[1/8,7/8]]), Matrix(QQ,[[1/2,1/2],[1/4,3/4]])];
#bert1 = [Matrix(QQ,[[1/4,3/4],[1/8,7/8]]), Matrix(QQ,[[1/2,1/2],[0,1]])];
sortbert1 = sortproblist(list_probs(bert1,lengths=range(1,5),accept_states=(1,0)));

In [56]:
#0 -> 01, 1 -> 1
def psitest(string):
    newstr = [];
    for ch in string:
        if int(ch) == 0:
            newstr.append(0);
            newstr.append(1);
        elif int(ch) == 1:
            newstr.append(1);
    return newstr;

#binary expansion
def binexp(string):
    value = 0;
    for i in range(len(string)):
        value = value + string[i]/2^(i+1);
    return QQ(value);

In [57]:
for thing in sortbert1:
    print thing," ",binexp(psitest(thing[0]))," ",binexp(psitest(thing[0]))-thing[1]

((1,), 1/2)   1/2   0
((1, 1), 3/8)   3/4   3/8
((1, 1, 1), 11/32)   7/8   17/32
((1, 1, 1, 1), 43/128)   15/16   77/128
((0, 1, 1, 1), 85/256)   15/32   35/256
((0, 1, 1), 21/64)   7/16   7/64
((1, 0, 1, 1), 83/256)   23/32   101/256
((0, 0, 1, 1), 165/512)   23/64   19/512
((0, 1), 5/16)   3/8   1/16
((1, 0, 1), 19/64)   11/16   25/64
((1, 1, 0, 1), 75/256)   27/32   141/256
((0, 1, 0, 1), 149/512)   27/64   67/512
((0, 0, 1), 37/128)   11/32   7/128
((1, 0, 0, 1), 147/512)   43/64   197/512
((0, 0, 0, 1), 293/1024)   43/128   51/1024
((0,), 1/4)   1/4   0
((1, 0), 3/16)   5/8   7/16
((1, 1, 0), 11/64)   13/16   41/64
((1, 1, 1, 0), 43/256)   29/32   189/256
((0, 1, 1, 0), 85/512)   29/64   147/512
((0, 1, 0), 21/128)   13/32   31/128
((1, 0, 1, 0), 83/512)   45/64   277/512
((0, 0, 1, 0), 165/1024)   45/128   195/1024
((0, 0), 5/32)   5/16   5/32
((1, 0, 0), 19/128)   21/32   65/128
((1, 1, 0, 0), 75/512)   53/64   349/512
((0, 1, 0, 0), 149/1024)   53/128   275/1024
((0, 0, 0), 37/

In [58]:
rholist = [];
for thing in sortbert1:
    rholist.append((thing[0],binexp(psitest(thing[0]))));
print sorted(rholist,key=lambda x:x[1],reverse=True)

[((1, 1, 1, 1), 15/16), ((1, 1, 1, 0), 29/32), ((1, 1, 1), 7/8), ((1, 1, 0, 1), 27/32), ((1, 1, 0, 0), 53/64), ((1, 1, 0), 13/16), ((1, 1), 3/4), ((1, 0, 1, 1), 23/32), ((1, 0, 1, 0), 45/64), ((1, 0, 1), 11/16), ((1, 0, 0, 1), 43/64), ((1, 0, 0, 0), 85/128), ((1, 0, 0), 21/32), ((1, 0), 5/8), ((1,), 1/2), ((0, 1, 1, 1), 15/32), ((0, 1, 1, 0), 29/64), ((0, 1, 1), 7/16), ((0, 1, 0, 1), 27/64), ((0, 1, 0, 0), 53/128), ((0, 1, 0), 13/32), ((0, 1), 3/8), ((0, 0, 1, 1), 23/64), ((0, 0, 1, 0), 45/128), ((0, 0, 1), 11/32), ((0, 0, 0, 1), 43/128), ((0, 0, 0, 0), 85/256), ((0, 0, 0), 21/64), ((0, 0), 5/16), ((0,), 1/4)]


In [43]:
pathlist(bert1,(1,0),(1,0))

path [0, 0, 0] has prob 1/8
path [0, 1, 0] has prob 1/16


In [47]:
bert2 = [Matrix(QQ,[[0,1,0,0],[0,0,1,0],[1/2,1/2,0,0],[0,1,0,0]]),
         Matrix(QQ,[[1/2,0,0,1/2],[1/2,0,1/2,0],[0,0,0,0],[1/2,0,0,1/2]])];
sortbert2 = sortproblist(list_probs(bert2,lengths=range(1,9),accept_states=(1,0,0,0)));

In [62]:
pathlist(bert2,(0,1,1,0,1),(1,0,0,0))

path [0, 1, 0, 0, 1, 0] has prob 1/8
path [0, 1, 0, 3, 1, 0] has prob 1/8


In [63]:
pathlist(bert2,(0,1,1,1),(1,0,0,0))

path [0, 1, 0, 0, 0] has prob 1/8
path [0, 1, 0, 3, 0] has prob 1/8


In [64]:
pathlist(bert2,(0,1,0,1,1),(1,0,0,0))

path [0, 1, 2, 0, 0, 0] has prob 1/16
path [0, 1, 0, 1, 0, 0] has prob 1/8
path [0, 1, 2, 1, 0, 0] has prob 1/16
path [0, 1, 2, 0, 3, 0] has prob 1/16


In [113]:
#testing Bertoni's 1977 PFA
bert77 = [Matrix(QQ,[[1/2,1/2],[1/4,3/4]]), Matrix(QQ,[[1/2,1/2],[0,1]])];
#sortbert77 = sortproblist(list_probs(bert77,lengths=range(1,11),accept_states=(0,1)));

In [108]:
#r(x) from 1977 paper
#thestr must be an iterable
def bert_r(thestr,base):
    value = 0;
    m = len(thestr);
    for j in range(1,m+1):
        value = value + thestr[j-1]*base^(j-1-m);
    return value;

In [116]:
for thing in S[0]:
    pro = accprob(bert77,thing,accept_states=(0,1));
    be = bert_r(psitest(thing),2);
    if pro != be:
        print thing,psitest(thing),"  prob(orig)",pro,"  psi(orig)",be

In [34]:
for s in SW.keys():
    if len(SW[s]) == 0:
        continue;
    valid = False;
    for wit in SW[s]:
        if isvalidpfa(wit[0]) == False:
            print s,wit
            SW[s].remove(wit);
            continue;
        else:
            valid = True;
    if valid == False:
        print s,SC[s]#,SW[s]
        SC[s] = 100; #ridiculous value meaning we have yet to determine
        SW[s] = [];

In [35]:
for s in SC.keys():

        b = [1,0,0];
        f = [0,0,1];
        #f = list(reversed(f));
        wits = findwitness(some3x4_2,s,b,f);
        if wits != None:
            print s,wits,addonewit(wits,s,b,f);
            print "";
            print flip(s),switchmats(wits),addonewit(switchmats(wits),flip(s),b,f);
            print "";

(1, 0, 1, 0, 1, 1, 1, 0, 0) [[1/4   0 3/4]
[  1   0   0]
[  0   1   0], [  0 3/4 1/4]
[1/4 3/4   0]
[3/4 1/4   0]] 2

(0, 1, 0, 1, 0, 0, 0, 1, 1) [[  0 3/4 1/4]
[1/4 3/4   0]
[3/4 1/4   0], [1/4   0 3/4]
[  1   0   0]
[  0   1   0]] 2

(1, 0, 0, 1, 1, 1, 1, 0, 1) [[1/4 3/4   0]
[1/4   0 3/4]
[  0   1   0], [3/4   0 1/4]
[  0 1/4 3/4]
[3/4 1/4   0]] 2

(0, 1, 1, 0, 0, 0, 0, 1, 0) [[3/4   0 1/4]
[  0 1/4 3/4]
[3/4 1/4   0], [1/4 3/4   0]
[1/4   0 3/4]
[  0   1   0]] 2

(1, 1, 0, 1, 0, 1, 1, 1) [[1/4 3/4   0]
[  1   0   0]
[  1   0   0], [  0 1/4 3/4]
[3/4   0 1/4]
[1/4   0 3/4]] 2

(0, 0, 1, 0, 1, 0, 0, 0) [[  0 1/4 3/4]
[3/4   0 1/4]
[1/4   0 3/4], [1/4 3/4   0]
[  1   0   0]
[  1   0   0]] 2

(1, 0, 1, 1, 1, 0, 0, 1, 1) [[1/4 3/4   0]
[1/4   0 3/4]
[  1   0   0], [3/4 1/4   0]
[  0 1/4 3/4]
[1/4   0 3/4]] 2

(0, 1, 0, 0, 0, 1, 1, 0, 0) [[3/4 1/4   0]
[  0 1/4 3/4]
[1/4   0 3/4], [1/4 3/4   0]
[1/4   0 3/4]
[  1   0   0]] 2

(0, 0, 0, 0, 1, 1, 1, 0, 0) [[1/4 3/4   0]
[  0   0   1]
[  1 

In [52]:
#really inefficient (for larger sets of strings)
c = 0;
for s in S[9].difference(lookup(SC,3,9,less=True)):
    if len(SW[s]) > 0:
        continue;
    c = c + 1;
    if c < 0:
        continue;
    if c > 50:
        break;
    sys.stdout.write("%d\r" % (c) );
    v = [1,0,0];
    f = [0,0,1];
    f = list(reversed(f));
    wits = findwitness(some3x4[::5],s,f);
    #wits = findwitness(half3x3[269502::91],s,v,f);
    #wits = findwitness(test2x15[::13],s,f);
    if wits != None:
        print s,wits,addonewit(wits,s,v,f);
        print "";
        print flip(s),switchmats(wits),addonewit(switchmats(wits),flip(s),v,f);
        print "";
print S[9].difference(lookup(SC,3,9,less=True)).cardinality();

(0, 0, 1, 1, 0, 1, 0, 0, 1) [[1/4 1/2 1/4]
[  1   0   0]
[  0   1   0], [  0   0   1]
[1/4 1/4 1/2]
[  0   1   0]] 2

(1, 1, 0, 0, 1, 0, 1, 1, 0) [[  0   0   1]
[1/4 1/4 1/2]
[  0   1   0], [1/4 1/2 1/4]
[  1   0   0]
[  0   1   0]] 2

(0, 1, 0, 0, 1, 1, 0, 1, 0) [[  0   0   1]
[1/2 1/4 1/4]
[  0   1   0], [1/4 1/2 1/4]
[1/4 1/4 1/2]
[1/2 1/4 1/4]] 2

(1, 0, 1, 1, 0, 0, 1, 0, 1) [[1/4 1/2 1/4]
[1/4 1/4 1/2]
[1/2 1/4 1/4], [  0   0   1]
[1/2 1/4 1/4]
[  0   1   0]] 2

(1, 1, 1, 0, 0, 1, 0, 1, 1) [[1/4 1/4 1/2]
[  1   0   0]
[1/4 1/4 1/2], [  0   0   1]
[  1   0   0]
[1/2 1/4 1/4]] 2

(0, 0, 0, 1, 1, 0, 1, 0, 0) [[  0   0   1]
[  1   0   0]
[1/2 1/4 1/4], [1/4 1/4 1/2]
[  1   0   0]
[1/4 1/4 1/2]] 2

(1, 0, 1, 1, 1, 0, 1, 0, 1) [[1/2 1/4 1/4]
[  0   1   0]
[  1   0   0], [1/4 1/4 1/2]
[1/4 1/2 1/4]
[  1   0   0]] 2

(0, 1, 0, 0, 0, 1, 0, 1, 0) [[1/4 1/4 1/2]
[1/4 1/2 1/4]
[  1   0   0], [1/2 1/4 1/4]
[  0   1   0]
[  1   0   0]] 2

54


In [53]:
savewits()

In [10]:
vex = [(1,0),(1,2)]
Tuples(vex,5).list()

[[(1, 0), (1, 0), (1, 0), (1, 0), (1, 0)],
 [(1, 2), (1, 0), (1, 0), (1, 0), (1, 0)],
 [(1, 0), (1, 2), (1, 0), (1, 0), (1, 0)],
 [(1, 2), (1, 2), (1, 0), (1, 0), (1, 0)],
 [(1, 0), (1, 0), (1, 2), (1, 0), (1, 0)],
 [(1, 2), (1, 0), (1, 2), (1, 0), (1, 0)],
 [(1, 0), (1, 2), (1, 2), (1, 0), (1, 0)],
 [(1, 2), (1, 2), (1, 2), (1, 0), (1, 0)],
 [(1, 0), (1, 0), (1, 0), (1, 2), (1, 0)],
 [(1, 2), (1, 0), (1, 0), (1, 2), (1, 0)],
 [(1, 0), (1, 2), (1, 0), (1, 2), (1, 0)],
 [(1, 2), (1, 2), (1, 0), (1, 2), (1, 0)],
 [(1, 0), (1, 0), (1, 2), (1, 2), (1, 0)],
 [(1, 2), (1, 0), (1, 2), (1, 2), (1, 0)],
 [(1, 0), (1, 2), (1, 2), (1, 2), (1, 0)],
 [(1, 2), (1, 2), (1, 2), (1, 2), (1, 0)],
 [(1, 0), (1, 0), (1, 0), (1, 0), (1, 2)],
 [(1, 2), (1, 0), (1, 0), (1, 0), (1, 2)],
 [(1, 0), (1, 2), (1, 0), (1, 0), (1, 2)],
 [(1, 2), (1, 2), (1, 0), (1, 0), (1, 2)],
 [(1, 0), (1, 0), (1, 2), (1, 0), (1, 2)],
 [(1, 2), (1, 0), (1, 2), (1, 0), (1, 2)],
 [(1, 0), (1, 2), (1, 2), (1, 0), (1, 2)],
 [(1, 2), (